<a href="https://colab.research.google.com/github/CaptainAmu/Reinforcement-Learning-Tutorial/blob/main/notebooks/unit1/unit1_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies & Install packages

In [1]:
%%html
<video controls autoplay><source src="https://huggingface.co/sb3/ppo-LunarLander-v2/resolve/main/replay.mp4" type="video/mp4"></video>

In [2]:
!apt install swig cmake

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [3]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

  Using cached swig-4.3.1.post0-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (3.5 kB)
  Using cached pygame-2.1.3.tar.gz (12.8 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [8]:
# Use some of the newest version of packages

!pip install pygame==2.5.2 -q # -q quiet install
!pip install box2d-py==2.3.5 -q
!pip install gymnasium>=1.0.0 -q
!pip install stable-baselines3==2.0.0a5 -q
!pip install huggingface_sb3 -q

In [9]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

```pygame==2.5.2```

一个游戏开发框架，RL 里常用来渲染环境画面（比如 LunarLander 的小飞船动起来）。在 gymnasium[box2d] 里是必需的。

```box2d-py==2.3.5```

Box2D 是一个 2D 物理引擎，用来模拟重力、碰撞等。LunarLander-v2 就是基于 Box2D 实现的。

```gymnasium```

强化学习的标准环境库（OpenAI Gym 的继承版本）。提供各种环境：LunarLander-v2、CartPole-v1、Atari 等。

```stable-baselines3==2.0.0a5```

常用的深度强化学习算法库（PPO、DQN、A2C 等）。你训练智能体时用的核心库。

```huggingface_sb3```

Hugging Face 提供的扩展包，方便把训练好的 RL 模型上传/下载到 Hugging Face Hub。类似“模型仓库管理工具”。[链接文字](https://)

---
## Import Packages

In [10]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [43]:
import gymnasium
print(f'Using Gymnasium version {gymnasium.__version__}')

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO, DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

Using Gymnasium version 0.28.1


---
## Random policy

In [12]:
import gymnasium as gym

# Create environment and initial observation
env = gym.make("LunarLander-v2")
observation, info = env.reset()

# Sample actions and update environment
for _ in range(20):
  action = env.action_space.sample()
  print(f'Action taken: {action}')
  observation, reward, terminated, truncated, info = env.step(action)

  if terminated or truncated:
    print("Environment is reset!")
    observation, info = env.reset()

env.close()

Action taken: 3
Action taken: 2
Action taken: 1
Action taken: 3
Action taken: 3
Action taken: 2
Action taken: 1
Action taken: 2
Action taken: 2
Action taken: 3
Action taken: 1
Action taken: 0
Action taken: 3
Action taken: 2
Action taken: 0
Action taken: 1
Action taken: 3
Action taken: 3
Action taken: 2
Action taken: 0


## Investigate the environment

In [32]:
env = gym.make("LunarLander-v2")
print(f'Action space, {env.action_space}')
print(f'Action space sample: {env.action_space.sample()}')
print(f'Observation space, {env.observation_space}')
print(f'Observation space sample: {env.observation_space.sample()}')

Action space, Discrete(4)
Action space sample: 3
Observation space, Box([-90.        -90.         -5.         -5.         -3.1415927  -5.
  -0.         -0.       ], [90.        90.         5.         5.         3.1415927  5.
  1.         1.       ], (8,), float32)
Observation space sample: [ 65.4944     -46.4049       0.89869606   2.755113     2.9383454
   0.3265673    0.15599686   0.94359624]


The observation space (8 entries) describes the x-coord, y-coord, x-velocity, y-velocity, angle, angular-velocity, left_leg_on_ground, right_leg_on_ground.

In [36]:
env = make_vec_env("LunarLander-v2", n_envs = 16) # 16 envs in parallel

## Create a Model

In [47]:
# PPO Model
model_PPO = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose = 1
)

# DQN Model
model_DQN = DQN(
    policy = 'MlpPolicy',
    env = env,
    batch_size = 64,
    gamma = 0.999,
    learning_rate = 0.00025
    )

Using cpu device


## Training a agent




In [40]:
model_PPO.learn(total_timesteps = 100000)
model_PPO.save("PPO-Lunarlander-v2")

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 93.2     |
|    ep_rew_mean     | -178     |
| time/              |          |
|    fps             | 3322     |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 16384    |
---------------------------------


In [48]:
model_DQN.learn(total_timesteps = 100000)
model_DQN.save("DQN-Lunarlander-v2")

## Evaluate an agent

In [53]:
def evaluate(agent):
  '''Evaluate the performance of a trained agent in LunarLander-v2'''
  eval_env = Monitor(gym.make("LunarLander-v2", render_mode = 'rgb_array'))
  mean_reward, std_reward = evaluate_policy(agent, eval_env, n_eval_episodes = 10, deterministic = True)
  print(f'Mean reward={mean_reward}+-{std_reward}')
  return(mean_reward, std_reward)


evaluate(model_DQN)

Mean reward=-163.6997762+-79.93649502872515


(np.float64(-163.6997762), np.float64(79.93649502872515))

## Pushing a model to the Hugging Face Hub
### 1. Create repo (with access token) and login to it.

In [54]:
notebook_login()
!git config --global credential.helper store

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### 2. Push model to Hub

In [55]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from huggingface_sb3 import package_to_hub

In [56]:
model = model_DQN
repo_id = 'ShuchengLi/dqn-LunarLander-v2'
model_name = 'dqn-LunarLander-v2'
env_id = 'LunarLander-v2'
model_architecture = 'DQN'
commit_message = 'Upload DQN LunarLander-v2 trained agent'
eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode = 'rgb_array'))])

package_to_hub(model=model,
               model_name=model_name,
               model_architecture=model_architecture,
               env_id=env_id,
               eval_env=eval_env,
               repo_id=repo_id,
               commit_message=commit_message)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Saving video to /tmp/tmphl51_7ox/-step-0-to-step-1000.mp4


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"


Moviepy - Building video /tmp/tmphl51_7ox/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmphl51_7ox/-step-0-to-step-1000.mp4



...

  .../tmpk_t6so28/dqn-LunarLander-v2.zip: 100%|##########|  109kB /  109kB            

  /tmp/tmpk_t6so28/replay.mp4           : 100%|##########|  188kB /  188kB            

  /tmp/tmpk_t6so28/replay.mp4           : 100%|##########|  188kB /  188kB            

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/ShuchengLi/dqn-LunarLander-v2/tree/main/


CommitInfo(commit_url='https://huggingface.co/ShuchengLi/dqn-LunarLander-v2/commit/ad1f94b9d398b23b307eff5b573affc46b41f9a7', commit_message='Upload DQN LunarLander-v2 trained agent', commit_description='', oid='ad1f94b9d398b23b307eff5b573affc46b41f9a7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ShuchengLi/dqn-LunarLander-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='ShuchengLi/dqn-LunarLander-v2'), pr_revision=None, pr_num=None)